# **Machine-to-Machine (M2M) Landsat Download: Bands, Bundles, and Band Groups**

### **What is M2M?**
EarthExplorer (EE) serves as a key data access portal, offering a range of tools for searching, discovering, and downloading data and metadata from the USGS Earth Resources Observation and Science (EROS) data repository. The Machine-to-Machine (M2M) API allows users to search and retrieve download URLS and metadata from the EROS archive by using programming languages like Python or PHP. Users can create JSON structures to pass to M2M endpoints, subsequently receiving JSON responses in return.

### **Requesting Access**
Users will need to log on to their [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) accounts to view view the M2M documentation. To submit download requests through M2M, ERS users must be authorized. 
Steps to request M2M access:
1. Login to [**EROS Registration Service (ERS)**](https://ers.cr.usgs.gov/) (See [**How to Create an ERS Account**](https://www.youtube.com/watch?v=Ut6kxbuP_nk))
2. Select the **Access Request** menu
3. From the **Access Controls** page, use the **Request Access** button
4. In the **Access Type** selector, choose **Access to EE's Machine to Machine interface (MACHINE)**, then complete the questions

<div class="alert alert-warning"><h4>
The USGS/EROS User Services team assesses these requests. Contact USGS EROS User Services for more information: <a href="https://www.usgs.gov/staff-profiles/usgs-eros-customer-services" target="_blank">custserv@usgs.gov</a> and visit the <a href="https://www.usgs.gov/centers/eros/science/earthexplorer-help-index#ers" target="_blank">EarthExplorer Help Index</a>   </h4></div>


### **Use Case Scenario**
This notebook illustrates how to download Landsat Collection 2 ***bands***, ***bundles***, and ***band groups*** using the M2M API. In this context, ***"bands"*** refer to individual band files in **.TIF** format, ***"bundles"*** include all files for a product(s) as **.tar** files, and ***"band groups"*** include downloading all files for a given product(s) as individual files (unpacked from the .tar file within a bundle). The area of interest (AOI) location for used is Anchorage, Alaska.

You will be allowed to select the download ***filetype*** before proceeding.


After 1. [**Setting a File Download Type**](#setdownloadtype) his example shows the user how to:

2. [**Create Area of Interest GeoJSON file**](#createaoi)
3. [**Retrieve Scenes using the *scene-search* endpoint**](#retrievescenes)
4. [**Request a *scene-list-add***](#scenelistadd)
5. [**Get *download-options***](#downloadoptions)
6. [**Set *data-file-group* Ids**](#setdatafilegroups)
7. [**Select Products for Downloading**](#selectproducts)
8. [**Sending *download-request* and *download-retrieve* commands**](#downloadrequest)
9. [**Removing a scene-list with *scene-list-remove***](#scenelistremove)
10. [**Logout of M2M endpoint**](#logout)

##### To view other M2M examples visit the [M2M Machine-to-Machine (M2M) API Example](https://m2m.cr.usgs.gov/api/docs/examples) page.


## **Setup**

### **Import necessary libraries**

In [1]:
import json
import requests
from getpass import getpass
import sys
import time
import re
import threading
import datetime
import os
import pandas as pd
import geopandas as gpd

import warnings
warnings.filterwarnings("ignore")

### **Define send request function**
This function sends a request to a M2M endpoint and returns the parsed JSON response.

Input parameters include:
- **endpoint_url** (*str*): The URL of the M2M endpoint
- **payload** (*dict*): The payload to be sent with the request

Returns:
- **dict**: Parsed JSON response

In [2]:
# Función para leer credenciales desde un archivo
def read_credentials(file_path):
    credentials = {}
    with open(file_path, 'r') as file:
        for line in file:
            if ':' in line:
                key, value = line.strip().split(':', 1)
                credentials[key] = value
    return credentials

# Definición de la Función `sendRequest`
def sendRequest(url, data, headers=None, exitIfNoResponse=True):
    json_data = json.dumps(data)
    
    if headers is None:
        response = requests.post(url, data=json_data)
    else:
        response = requests.post(url, data=json_data, headers=headers)
    
    try:
        httpStatusCode = response.status_code
        if response is None:
            print("No output from service")
            if exitIfNoResponse: sys.exit()
            else: return False
        
        output = json.loads(response.text)
        if output['errorCode'] is not None:
            print(output['errorCode'], "- ", output['errorMessage'])
            if exitIfNoResponse: sys.exit()
            else: return False
        
        if httpStatusCode == 404:
            print("404 Not Found")
            if exitIfNoResponse: sys.exit()
            else: return False
        elif httpStatusCode == 401:
            print("401 Unauthorized")
            if exitIfNoResponse: sys.exit()
            else: return False
        elif httpStatusCode == 400:
            print("Error Code", httpStatusCode)
            if exitIfNoResponse: sys.exit()
            else: return False
        else:
            return output['data']
    except Exception as e:
        print("An error occurred: ", str(e))
        if exitIfNoResponse: sys.exit()
        else: return False

### **Define download function**

In [3]:
def downloadFile(url):
    sema.acquire()
    try:
        response = requests.get(url, stream=True)
        disposition = response.headers['content-disposition']
        filename = re.findall("filename=(.+)", disposition)[0].strip("\"")
        print(f"    Downloading: {filename}...")
        
        open(os.path.join(data_dir, filename), 'wb').write(response.content)
        sema.release()
    except Exception as e:
        print(f"\nFailed to download from {url}. Will try to re-download.")
        sema.release()
        runDownload(threads, url)

### **Define *runDownload* function**
An additonal function that uses the ***downloadFile*** function above to allow multiple files to be downloaded simultaneously.

In [4]:
def runDownload(threads, url):
    thread = threading.Thread(target=downloadFile, args=(url,))
    threads.append(thread)
    thread.start()

### **Set up output directory**
This section creates a ***data*** and ***utils*** directories.

In [5]:
data_dir = 'data'
utils_dir = 'utils'
dirs = [ data_dir, utils_dir]

for d in dirs:
        if not os.path.exists(d): 
            try: 
                os.makedirs(d)
                print(f"Directory '{d}' created successfully.") 
            except OSError as e: 
                print(f"Error creating directory '{d}': {e}") 
        else: 
            print(f"Directory '{d}' already exists.") 

Directory 'data' already exists.
Directory 'utils' already exists.


### **<span style="color:#DF0101">Deleting files in *data* directory</span>**

<span style="color:#DF0101">This script is designed to clear all files in the data directory, providing users with a clean slate to examine the types of files downloaded with each download type. Uncomment this section if the **data** directory is not empty.</span> 

In [6]:
# # List all files in the directory
# files = os.listdir(data_dir)

# # Loop through the files and delete them one by one
# for file in files:
#     file_path = os.path.join(data_dir, file)
    
#     try:
#         # Check if the file exists and is a regular file (not a directory)
#         if os.path.isfile(file_path):
#             os.remove(file_path)
#             print(f"Deleted file: {file_path}")
#         else:
#             print(f"Skipped non-file: {file_path}")
#     except Exception as e:
#         print(f"Error deleting file: {file_path} - {str(e)}")

# # Print a message indicating the process is complete
# print("File deletion complete.")

### **Set download thread parameters**

In [7]:
maxthreads = 5 # Threads count for downloads
sema = threading.Semaphore(value=maxthreads)
label = datetime.datetime.now().strftime("%Y%m%d_%H%M%S") # Customized label using date time
threads = []

## **User Input**
#### **Login by entering [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) *username* and *password* when prompted**

In [8]:
# Leer credenciales desde el archivo
cred_file_path = 'C:\Workspace\m2m_landsat_bands_bundle_download\credenciales.txt'
credentials = read_credentials(cred_file_path)
username = credentials['username']
token = credentials['token']

# Generar la clave de API usando el endpoint login-token
serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
url = serviceUrl + "login-token"
payload = {"username": username, "token": token}
response = requests.post(url, json=payload)

if response.status_code == 200:
    apiKey = response.json()['data']
    print('\nLogin Successful, API Key Received!')
    headers = {'X-Auth-Token': apiKey}
else:
    print("\nLogin was unsuccessful. Please check your token and try again.")
    print(response.json())  # Print the error message from the response
    raise Exception("Login failed")

# Print the API key only if login was successful
if apiKey:
    print("\nAPI Key: " + apiKey + "\n")


Login Successful, API Key Received!

API Key: eyJjaWQiOjEzNDY3MjYsInMiOiIxNzIyOTE2OTg3IiwiciI6Nzk2LCJwIjpbInVzZXIiLCJkb3dubG9hZCIsIm9yZGVyIl19



#### **OR**

#### **Login by setting [EROS Registration System (EROS)](https://ers.cr.usgs.gov/https://ers.cr.usgs.gov/) *username* and *password***

In [9]:
# username = ""
# password = ""

In [10]:
# print("Logging in...\n")
    
# serviceUrl = "https://m2m.cr.usgs.gov/api/api/json/stable/"
# login_payload = {'username' : username, 'password' : password}
    
# apiKey = sendRequest(serviceUrl + "login", login_payload)
    
# print("API Key: " + apiKey + "\n")

## **1. Set File Download Type <a id="setdownloadtype"></a>**

#### <span style="color:#DF0101">Uncomment your preferred download type before proceeding.</span> 

- #### **Bundle**
For downloading bundles of Landsat scenes (downloads data for select scenes as ***.tar*** files).

In [11]:
# fileType = 'bundle'

- #### **Band**
For downloading individual band files as ***.TIF*** files.
Must include ***bandNames*** for each band e.g. ***'SR_B2'*** will search for Band 2 from the Surface Reflectance Products.

In [12]:
fileType = 'band'
bandNames = {'SR_B3', 'SR_B5'}

- #### **Band Group**
Downloads all bands for a given product as individual files.

In [13]:
# fileType = 'band_group'

## **2. Create an Area of Interest (AOI) GeoJSON Text File** <a id="createaoi"></a>

Polygon coordinates are arranged in the sequence NE, NW, SW, SE, NE, representing all four corners, with an extra NE point to conclude and close the polygon.

In [14]:
from geojson import Polygon, Feature, FeatureCollection, dump

polygon = Polygon([[[-68.00847232151489, 9.993051075414456],  # Esquina inferior izquierda
                    [-68.00741959094407, 10.334374973060754],  # Esquina superior izquierda
                    [-67.49442572538628, 10.332388252838966],  # Esquina superior derecha
                    [-67.49602212224453, 9.991131292108372],   # Esquina inferior derecha
                    [-68.00847232151489, 9.993051075414456]]])  # Cerrando el polígono

features = []
features.append(Feature(geometry=polygon, properties={"name": "Lago de Valencia Area", "country": "Venezuela"}))

# Crear la colección de características
feature_collection = FeatureCollection(features)

# Guardar en un archivo GeoJSON
with open('./utils/Lago_de_Valencia_Venezuela_aoi.geojson', 'w') as f:
    dump(feature_collection, f)

- #### Open the GeoJSON file as a geopandas dataframe

In [15]:
aoi_geodf =  gpd.read_file('./utils/Lago_de_Valencia_Venezuela_aoi.geojson') #aoi

- #### View the Coordinate Reference System (CRS)
The CRS used is the World Geodetic System 1984 with units in degrees.

In [16]:
aoi_geodf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

### **Plot AOI using Folium**
The ***folium*** module is used for creating interactive web maps.

In [17]:
import folium
m = folium.Map(location=[aoi_geodf.centroid.y[0], aoi_geodf.centroid.x[0]], zoom_start=8, tiles="openstreetmap",\
              width="90%",height="90%", attributionControl=0) #add n estimate of where the center of the polygon would be located\
                                        #for the location [latitude longitude]

In [18]:
for _, r in aoi_geodf.iterrows():
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "blue"})
    geo_j.add_to(m)
m

## **3. Retrieve Scenes using the [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) endpoint** <a id="retrievescenes"></a>

The [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) is used for searching for scenes within a dataset collection. A ***'datasetName'*** parameter is required; it is used to identify the dataset to search for. In this case, we will be downloading bands from the ***Landsat 8-9 Operational Land Imager and Thermal Infrared Sensor Collection 2 Level-2*** with a datasetName: ***landsat_ot_c2_l2***.

In [19]:
datasetName = 'landsat_ot_c2_l2'

<div class="alert alert-info">
    <h4>To find the <b><i>datasetName</i></b> for other collections of Landsat data:</h4>
    <ol>
        <li>
            Run a request with the <a href="https://m2m.cr.usgs.gov/api/docs/reference/#dataset-search" target="_blank">
                <b><i>dataset-search</i></b>
            </a> request and extract the <b><i>'datasetAlias'</i></b> field. No input parameters are necessary!
        </li>
        <li>
            Run the <b><i>dataset-search</i></b> endpoint using the
            <a href="https://m2m.cr.usgs.gov/api/test/json/" target="_blank">
                <b><i>Machine-to-Machine (M2M) Test Page</i></b>
            </a>.
        </li>
    </ol>
</div>


### **3.1 Setup [*scene-filters*](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter):**

The [***spatialFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilter), [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) and [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) are [***scene-filters***](https://m2m.cr.usgs.gov/api/docs/datatypes/#sceneFilter) used below to identify the scenes matching the filter criteria using [**scene-search**](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search).

- #### ***spatialFilter***
There are two ***filterType's***:

> - [***geojson***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilterGeoJson) which needs a ***['geoJson'](https://m2m.cr.usgs.gov/api/docs/datatypes/#geoJson)*** field name labeled with a geometric ***'type'*** e.g. ***'Polygon'*** and includes the coordinate array of the polygon:

~~~~
        spatialFilter = {'filterType' : 'geojson',
                         'geoJson' : {'type': 'Polygon',\ 
                                      'coordinates': [[[-148.9555, 61.4834],\
                                                       [-150.9495, 61.4834],\
                                                       [-150.9495, 61.0031],\
                                                       [-148.9555, 61.0031],\
                                                       [-148.9555, 61.4834]]]}}

~~~~

> - [***mbr***](https://m2m.cr.usgs.gov/api/docs/datatypes/#spatialFilterMbr) (Minimum Bounding Rectangle) where the user should include the ***'lowerLeft'*** (southwest point) and ***'upperRight'*** (northeast point) of the rectangle as ***{'latitude': 61.4834, 'longitude' : -148.9555}*** coordinates:

```
        spatialFilter =  {'filterType' : 'mbr',
                    'lowerLeft' : {'latitude' : 61.4834,\
                                   'longitude' : -148.9555},
                   'upperRight' : { 'latitude' : 61.0031,\
                                   'longitude' : -150.9495}}
```

In [20]:
# spatialFilter =  {'filterType' : 'mbr',
#                     'lowerLeft' : {'latitude' : aoi_geodf.bounds.miny[0],\
#                                    'longitude' : aoi_geodf.bounds.minx[0]},
#                    'upperRight' : { 'latitude' : aoi_geodf.bounds.maxy[0],\
#                                    'longitude' : aoi_geodf.bounds.maxx[0]}}

spatialFilter = {
    'filterType': 'geojson',
    'geoJson': {
        'type': 'Polygon',
        'coordinates': [
            [
                [-68.00847232151489, 9.993051075414456],  # Esquina inferior izquierda
                [-68.00741959094407, 10.334374973060754],  # Esquina superior izquierda
                [-67.49442572538628, 10.332388252838966],  # Esquina superior derecha
                [-67.49602212224453, 9.991131292108372],   # Esquina inferior derecha
                [-68.00847232151489, 9.993051075414456]    # Cerrando el polígono
            ]
        ]
    }
}

- #### **acquisitionFilter**
 The [***acquisitionFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#acquisitionFilter) is the ***'start'*** and ***'end'*** dates in ISO format (**yyyy-mm-dd**) for the time period of interest. In our case it will be the same as the ***temporalFilter*** used above.

In [21]:
temporalFilter = {'start' : '2024-01-01', 'end' : '2024-08-06'}

- #### ***cloudCoverFilter***
To filter scenes by cloud cover, you can use the [***cloudCoverFilter***](https://m2m.cr.usgs.gov/api/docs/datatypes/#cloudCoverFilter) and specify a ***'min'*** and ***'max'*** percentage (%) value.

In [22]:
cloudCoverFilter = {'min' : 0, 'max' : 20}

### **3.2 Combine all parameters into a *payload* used for sending requests:**

In [23]:
search_payload = {
    'datasetName' : datasetName,
    'sceneFilter' : {
        'spatialFilter' : spatialFilter,
        'acquisitionFilter' : temporalFilter,
        'cloudCoverFilter' : cloudCoverFilter
    }
}

In [24]:
search_payload

{'datasetName': 'landsat_ot_c2_l2',
 'sceneFilter': {'spatialFilter': {'filterType': 'geojson',
   'geoJson': {'type': 'Polygon',
    'coordinates': [[[-68.00847232151489, 9.993051075414456],
      [-68.00741959094407, 10.334374973060754],
      [-67.49442572538628, 10.332388252838966],
      [-67.49602212224453, 9.991131292108372],
      [-68.00847232151489, 9.993051075414456]]]}},
  'acquisitionFilter': {'start': '2024-01-01', 'end': '2024-08-06'},
  'cloudCoverFilter': {'min': 0, 'max': 20}}}

### **3.3 Search for Scenes:**

In [25]:
scenes = sendRequest(serviceUrl + "scene-search", search_payload, headers)

In [26]:
pd.json_normalize(scenes['results'])

,browse,cloudCover,entityId,displayId,orderingId,metadata,hasCustomizedMetadata,publishDate,options.bulk,options.download,...,options.secondary,selected.bulk,selected.compare,selected.order,spatialBounds.type,spatialBounds.coordinates,spatialCoverage.type,spatialCoverage.coordinates,temporalCoverage.endDate,temporalCoverage.startDate
0,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",9,LC90050532024192LGN00,LC09_L2SP_005053_20240710_20240711_02_T1,None,[],None,2024-07-11 05:55:23-05,True,True,...,False,False,False,False,Polygon,"[[[-69.67569, 9.07001], [-69.67569, 11.16632],...",Polygon,"[[[-69.67569, 9.43134], [-67.98394, 9.07001], ...",2024-07-10 00:00:00,2024-07-10 00:00:00
1,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",11,LC90040532024105LGN00,LC09_L2SP_004053_20240414_20240415_02_T1,None,[],None,2024-04-15 06:02:58-05,True,True,...,False,False,False,False,Polygon,"[[[-68.12218, 9.0703], [-68.12218, 11.16561], ...",Polygon,"[[[-68.12218, 9.43054], [-66.42974, 9.0703], [...",2024-04-14 00:00:00,2024-04-14 00:00:00
2,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",17,LC80050532024104LGN00,LC08_L2SP_005053_20240413_20240419_02_T1,None,[],None,2024-04-19 04:21:00-05,True,True,...,False,False,False,False,Polygon,"[[[-69.64785, 9.06901], [-69.64785, 11.1668], ...",Polygon,"[[[-69.64785, 9.43143], [-67.95608, 9.06901], ...",2024-04-13 00:00:00,2024-04-13 00:00:00
3,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",8,LC90050532024080LGN00,LC09_L2SP_005053_20240320_20240321_02_T1,None,[],None,2024-03-21 08:56:10-05,True,True,...,False,False,False,False,Polygon,"[[[-69.68006, 9.07], [-69.68006, 11.16545], [-...",Polygon,"[[[-69.68006, 9.43026], [-67.98822, 9.07], [-6...",2024-03-20 00:00:00,2024-03-20 00:00:00
4,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",6,LC90040532024073LGN00,LC09_L2SP_004053_20240313_20240315_02_T1,None,[],None,2024-03-15 08:15:18-05,True,True,...,False,False,False,False,Polygon,"[[[-68.12753, 9.07005], [-68.12753, 11.16562],...",Polygon,"[[[-68.12753, 9.43034], [-66.43557, 9.07005], ...",2024-03-13 00:00:00,2024-03-13 00:00:00
5,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",20,LC80040532024065LGN00,LC08_L2SP_004053_20240305_20240315_02_T1,None,[],None,2024-03-15 12:38:34-05,True,True,...,False,False,False,False,Polygon,"[[[-68.09937, 9.06907], [-68.09937, 11.16715],...",Polygon,"[[[-68.09937, 9.43164], [-66.40801, 9.06907], ...",2024-03-05 00:00:00,2024-03-05 00:00:00
6,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",11,LC90050532024064LGN00,LC09_L2SP_005053_20240304_20240306_02_T1,None,[],None,2024-03-06 00:57:15-06,True,True,...,False,False,False,False,Polygon,"[[[-69.66453, 9.0703], [-69.66453, 11.16588], ...",Polygon,"[[[-69.66453, 9.43062], [-67.97284, 9.0703], [...",2024-03-04 00:00:00,2024-03-04 00:00:00
7,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",19,LC80050532024056LGN00,LC08_L2SP_005053_20240225_20240313_02_T1,None,[],None,2024-03-13 10:08:05-05,True,True,...,False,False,False,False,Polygon,"[[[-69.66431, 9.0687], [-69.66431, 11.16675], ...",Polygon,"[[[-69.66431, 9.43133], [-67.97288, 9.0687], [...",2024-02-25 00:00:00,2024-02-25 00:00:00
8,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",1,LC80040532024049LGN00,LC08_L2SP_004053_20240218_20240228_02_T1,None,[],None,2024-02-28 15:37:43-06,True,True,...,False,False,False,False,Polygon,"[[[-68.11749, 9.06873], [-68.11749, 11.16687],...",Polygon,"[[[-68.11749, 9.43154], [-66.4255, 9.06873], [...",2024-02-18 00:00:00,2024-02-18 00:00:00
9,"[{'id': '5fb4ba12d7ec307f', 'browseRotationEna...",11,LC90050532024048LGN00,LC09_L2SP_005053_20240217_20240218_02_T1,None,[],None,2024-02-18 14:53:52-06,True,True,...,False,False,False,False,Polygon,"[[[-69.64906, 9.07033], [-69.64906, 11.16605],...",Polygon,"[[[-69.64906, 9.43066], [-67.95778, 9.07033], ...",2024-02-17 00:00:00,2024-02-17 00:00:00


## **4. Request a [*scene-list-add*](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-add)**<a id="scenelistadd"></a>
A [***scene-list-add***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-add) compiles a list of scene or product IDs for download. A user defined  ***`listId`*** name is required.

### **4.1 Create a list of entityId's**
The [***scene-list-add***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-add) endpoint requires a list of **`entityIds`**, these are [Landsat Scene Identifiers](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_scene_id) that remain consistent for all levels and products available for each scene. Users can also enter a list of **`displayIds`** (also known as [Landsat Product Identifiers](https://www.usgs.gov/centers/eros/science/landsat-collection-2-data-dictionary#landsat_product_id)) in place of a list of **`entityIds`** but will need to label the **`idField`** as the **`'displayId'`**. Both the **`entityId`** and **`displayId`** are returned after running a [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search).

Here's an example of what the [***scene-list-add***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-add) payload would look like when a user wants to add scenes to a list using their entityId or displayId values:

- **entityIds**
~~~~
    {'listId': 'temp_landsat_ot_c2_l2_list',
     'idField': 'entityId',
     'entityIds': ['LC80680172020070LGN00', 'LC80680182020070LGN00'],
     'datasetName': 'landsat_ot_c2_l2'}
~~~~ 
- **displayIds**
~~~~ 
    {'listId': 'temp_landsat_ot_c2_l2_list',
     'idField': 'displayId',
     'entityIds': ['LC08_L2SP_068017_20200310_20200822_02_T1', 'LC08_L2SP_068018_20200310_20200822_02_T1'],
     'datasetName': 'landsat_ot_c2_l2'}
~~~~

Note the difference between the `'entityIds'` and `'idField'` parameters.
Also these examples use the results from the [***scene-search***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-search) submitted above.

<div class="alert alert-info" style="font-size: 14px;">
    <h4>
        Though it is possible to use a list of displayIds, you should optimize your workflow by using a list of <mark style="background-color: #ACD8E2; color: #0B4C5F;"><i>'entityIds'</i></mark> for the <mark style="background-color: #ACD8E2; color: #0B4C5F;"><i>'entityIds'</i></mark> parameter in the <a href="https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-add" target="_blank"><b><i>scene-list-add</i></b></a> request. This eliminates the need for individual entityId lookups for each displayId, significantly improving overall efficiency.
    </h4>
</div>

In [27]:
idField = 'entityId'

entityIds = []

for result in scenes['results']:
     # Add this scene to the list I would like to download if bulk is available
    if result['options']['bulk'] == True:
        entityIds.append(result[idField])
    
entityIds

['LC90050532024192LGN00',
 'LC90040532024105LGN00',
 'LC80050532024104LGN00',
 'LC90050532024080LGN00',
 'LC90040532024073LGN00',
 'LC80040532024065LGN00',
 'LC90050532024064LGN00',
 'LC80050532024056LGN00',
 'LC80040532024049LGN00',
 'LC90050532024048LGN00',
 'LC80040532024033LGN00',
 'LC90040532024025LGN00',
 'LC80050532024024LGN00',
 'LC80040532024017LGN00']

### **4.2 Add scenes to a list using [*scene-list-add*](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-add)**

In [28]:
listId = f"temp_{datasetName}_list" # customized list id
scn_list_add_payload = {
    "listId": listId,
    'idField' : idField,
    "entityIds": entityIds,
    "datasetName": datasetName
}
scn_list_add_payload

{'listId': 'temp_landsat_ot_c2_l2_list',
 'idField': 'entityId',
 'entityIds': ['LC90050532024192LGN00',
  'LC90040532024105LGN00',
  'LC80050532024104LGN00',
  'LC90050532024080LGN00',
  'LC90040532024073LGN00',
  'LC80040532024065LGN00',
  'LC90050532024064LGN00',
  'LC80050532024056LGN00',
  'LC80040532024049LGN00',
  'LC90050532024048LGN00',
  'LC80040532024033LGN00',
  'LC90040532024025LGN00',
  'LC80050532024024LGN00',
  'LC80040532024017LGN00'],
 'datasetName': 'landsat_ot_c2_l2'}

- #### The ***scene-list-add*** returns the number of scenes added to the list

In [29]:
count = sendRequest(serviceUrl + "scene-list-add", scn_list_add_payload, headers) 
count

54

- #### You can view the items added to the scene-list using [***scene-list-get***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-get)

In [30]:
sendRequest(serviceUrl + "scene-list-get", {'listId' : scn_list_add_payload['listId']}, headers) 

[{'entityId': 'LC90040532024025LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80050532024024LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80040532024017LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80040532023334LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC90040532023326LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80040532023302LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80050532023293LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80040532023286LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC90050532023285LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80040532023270LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC90050532023269LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80040532023238LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC90050532023237LGN00', 'datasetName': 'landsat_ot_c2_l2'},
 {'entityId': 'LC80040532

## **5. Get [*download-options*](https://m2m.cr.usgs.gov/api/docs/reference/#download-options)**<a id="downloadoptions"></a>
The [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options) request is used to discover downloadable products for each dataset. A ***'datasetName'*** parameter is required. The ***listId*** created above is used in the request.

In [31]:
download_opt_payload = {
    "listId": listId,
    "datasetName": datasetName
}

if fileType == 'band_group':
    download_opt_payload['includeSecondaryFileGroups'] = True

download_opt_payload

{'listId': 'temp_landsat_ot_c2_l2_list', 'datasetName': 'landsat_ot_c2_l2'}

In [32]:
products = sendRequest(serviceUrl + "download-options", download_opt_payload, headers)
pd.json_normalize(products)

,id,downloadName,displayId,entityId,datasetId,available,filesize,productName,productCode,bulkAvailable,downloadSystem,secondaryDownloads,fileGroups
0,5e83d14fec7cae84,None,LC08_L2SP_004053_20210516_20210525_02_T1,LC80040532021136LGN00,5e83d14f2fc39685,True,1020229051,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
1,6448198cc7b442a4,C2L2 Tile Product Files,LC08_L2SP_004053_20210516_20210525_02_T1,LC80040532021136LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D693,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
2,6448198c62023764,C2L2 Tile Product Files,LC08_L2SP_004053_20210516_20210525_02_T1,LC80040532021136LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D691,True,folder,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
3,632210d4770592cf,None,LC08_L2SP_004053_20210516_20210525_02_T1,LC80040532021136LGN00,5e83d14f2fc39685,False,1020229051,Landsat Collection 2 Level-2 Product Bundle,D806,False,dds_ms,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
4,5e83d14fec7cae84,None,LC08_L2SP_004053_20211210_20211216_02_T1,LC80040532021344LGN00,5e83d14f2fc39685,True,980149005,Landsat Collection 2 Level-2 Product Bundle,D694,True,ls_zip,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,632210d4770592cf,None,LC09_L2SP_005053_20240320_20240321_02_T1,LC90050532024080LGN00,5e83d14f2fc39685,False,1026059390,Landsat Collection 2 Level-2 Product Bundle,D806,False,dds_ms,"[{'id': '5f85f041a2ea6695', 'downloadName': No...",None
212,5e83d14fec7cae84,None,LC09_L2SP_005053_20240710_20240711_02_T1,LC90050532024192LGN00,5e83d14f2fc39685,False,1008987742,Landsat Collection 2 Level-2 Product Bundle,D694,False,ls_zip,"[{'id': '5f85f0419985f2aa', 'downloadName': No...",None
213,6448198cc7b442a4,C2L2 Tile Product Files,LC09_L2SP_005053_20240710_20240711_02_T1,LC90050532024192LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D693,True,folder,"[{'id': '5f85f0419985f2aa', 'downloadName': No...",None
214,6448198c62023764,C2L2 Tile Product Files,LC09_L2SP_005053_20240710_20240711_02_T1,LC90050532024192LGN00,5e83d14f2fc39685,True,0,Landsat Collection 2 Level-2 Band File,D691,True,folder,"[{'id': '5f85f0419985f2aa', 'downloadName': No...",None


## **6. Set [*dataset-file-groups*](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-file-groups) Ids** <a id="setdatafilegroups"></a>
This is necessary within this script for ***band_group*** downloads.
You can use the [***dataset-file-groups***](https://m2m.cr.usgs.gov/api/docs/reference/#dataset-file-groups) request to list all configured file groups for a dataset collection given the ***datasetName*** is provided.

In [33]:
filegroups = sendRequest(serviceUrl + "dataset-file-groups", {'datasetName' : datasetName}, headers)  
pd.json_normalize(filegroups['secondary'])

,5f85f041c828327a.ls_c2l2_sr_band.id,5f85f041c828327a.ls_c2l2_sr_band.color,5f85f041c828327a.ls_c2l2_sr_band.description,5f85f041c828327a.ls_c2l2_sr_band.displayOrder,5f85f041c828327a.ls_c2l2_sr_band.icon,5f85f041c828327a.ls_c2l2_sr_band.label,5f85f041c828327a.ls_c2l2_sr_band.files,5f85f041c828327a.ls_c2l2_st_band.id,5f85f041c828327a.ls_c2l2_st_band.color,5f85f041c828327a.ls_c2l2_st_band.description,5f85f041c828327a.ls_c2l2_st_band.displayOrder,5f85f041c828327a.ls_c2l2_st_band.icon,5f85f041c828327a.ls_c2l2_st_band.label,5f85f041c828327a.ls_c2l2_st_band.files
0,ls_c2l2_sr_band,None,Landsat Collection-2 Level-2 Surface Reflectan...,1,None,Level-2 Surface Reflectance Bands,"[{'name': 'ANG.txt', 'productIds': {'6453e8a2f...",ls_c2l2_st_band,None,Landsat Collection-2 Level-2 Surface Temperatu...,1,None,Level-2 Surface Temperature Bands,"[{'name': 'ANG.txt', 'productIds': {'6453e8a2f..."


#### ***fileGroupIds*** 
The ***landsat_ot_c2_l2*** dataset has two file groups:
- ***ls_c2l2_st_band*** (Landsat Collection-2 Level-2 Surface Temperature Bands)
- ***ls_c2l2_sr_band*** (Landsat Collection-2 Level-2 Surface Reflectance Bands)
    

In [34]:
fileGroupIds = {"ls_c2l2_sr_band"}

## **7. Select Products for Downloading**<a id="selectproducts"></a>
The script below uses the [**dataset**](https://m2m.cr.usgs.gov/api/docs/reference/#dataset) which is used to retrieve the dataset by id or name.
> Note that ***'products'*** are results from the endpoint [***download-options***](https://m2m.cr.usgs.gov/api/docs/reference/#download-options).

In [35]:

# Select products
print("Selecting products...")
downloads = []
if fileType == 'bundle':
    # Select bundle files
    print("    Selecting bundle files...")
    for product in products:        
        if product["bulkAvailable"] and product['downloadSystem'] != 'folder':               
            downloads.append({"entityId":product["entityId"], "productId":product["id"]})


elif fileType == 'band':
    # Select band files
    print("    Selecting band files...")
    for product in products:  
        if product["secondaryDownloads"] is not None and len(product["secondaryDownloads"]) > 0:
            for secondaryDownload in product["secondaryDownloads"]:
                for bandName in bandNames:
                    if secondaryDownload["bulkAvailable"] and bandName in secondaryDownload['displayId']:
                        downloads.append({"entityId":secondaryDownload["entityId"], "productId":secondaryDownload["id"]})


elif fileType == 'band_group':        
    # Get secondary dataset ID and file group IDs with the scenes
    print("    Checking for scene band groups and get secondary dataset ID and file group IDs with the scenes...")
    sceneFileGroups = []
    entityIds = []
    datasetId = None
    for product in products:  
        if product["secondaryDownloads"] is not None and len(product["secondaryDownloads"]) > 0:
            for secondaryDownload in product["secondaryDownloads"]:
                if secondaryDownload["bulkAvailable"] and secondaryDownload["fileGroups"] is not None:
                    if datasetId == None:
                        datasetId = secondaryDownload['datasetId']
                    for fg in secondaryDownload["fileGroups"]:                            
                        if fg not in sceneFileGroups:
                            sceneFileGroups.append(fg)
                        if secondaryDownload['entityId'] not in entityIds:
                            entityIds.append(secondaryDownload['entityId'])

    # Send dataset request to get the secondary dataset name by the dataset ID
    data_req_payload = {
        "datasetId": datasetId,
    }
    results = sendRequest(serviceUrl + "dataset", data_req_payload, headers)
    secondaryDatasetName = results['datasetAlias']

    # Add secondary scenes to a list
    secondaryListId = f"temp_{datasetName}_scecondary_list" # customized list id
    sec_scn_add_payload = {
        "listId": secondaryListId,
        "entityIds": entityIds,
        "datasetName": secondaryDatasetName
    }

    print("    Adding secondary scenes to list...")
    count = sendRequest(serviceUrl + "scene-list-add", sec_scn_add_payload, headers)    
    print("    Added", count, "secondary scenes\n")

    # Compare the provided file groups Ids with the scenes' file groups IDs
    if fileGroupIds:
        fileGroups = []
        for fg in fileGroupIds:
            fg = fg.strip() 
            if fg in sceneFileGroups:
                fileGroups.append(fg)
    else:
        fileGroups = sceneFileGroups
else:
    # Select all available files
    for product in products:        
        if product["bulkAvailable"]:
            if product['downloadSystem'] != 'folder':            
                downloads.append({"entityId":product["entityId"], "productId":product["id"]})
            if product["secondaryDownloads"] is not None and len(product["secondaryDownloads"]) > 0:
                for secondaryDownload in product["secondaryDownloads"]:
                    if secondaryDownload["bulkAvailable"]:
                        downloads.append({"entityId":secondaryDownload["entityId"], "productId":secondaryDownload["id"]})            

Selecting products...
    Selecting band files...


## **8. Send [*download-request*](https://m2m.cr.usgs.gov/api/docs/reference/#download-request)**<a id="downloadrequest"></a>

In [36]:
if fileType != 'band_group':
    download_req2_payload = {
        "downloads": downloads,
        "label": label
    }
else:
    if len(fileGroups) > 0:
        download_req2_payload = {
            "dataGroups": [
                {
                    "fileGroups": fileGroups,
                    "datasetName": secondaryDatasetName,
                    "listId": secondaryListId
                }
            ],
            "label": label
        }
    else:
        print('No file groups found')
        sys.exit()

print(f"Sending download request ...")
download_request_results = sendRequest(serviceUrl + "download-request", download_req2_payload, headers)
print(f"Done sending download request") 

if len(download_request_results['newRecords']) == 0 and len(download_request_results['duplicateProducts']) == 0:
    print('No records returned, please update your scenes or scene-search filter')
    sys.exit()


Sending download request ...
Done sending download request


### **Using [*download-retrieve*](https://m2m.cr.usgs.gov/api/docs/reference/#download-retrieve) to get URLs and start downloading:**

The [***download-retrieve***](https://m2m.cr.usgs.gov/api/docs/reference/#download-retrieve) endpoint returns a list of all available downloads and the ***runDownload*** function defined above is used for downloading the retrieved URLs.

In [37]:
# Attempt the download URLs
for result in download_request_results['availableDownloads']:       
    print(f"Get download url: {result['url']}\n" )
    runDownload(threads, result['url'])
    
preparingDownloadCount = len(download_request_results['preparingDownloads'])
preparingDownloadIds = []
if preparingDownloadCount > 0:
    for result in download_request_results['preparingDownloads']:  
        preparingDownloadIds.append(result['downloadId'])

    download_ret_payload = {"label" : label}                
    # Retrieve download URLs
    print("Retrieving download urls...\n")
    download_retrieve_results = sendRequest(serviceUrl + "download-retrieve", download_ret_payload, headers, False)
    if download_retrieve_results != False:
        print(f"    Retrieved: \n" )
        for result in download_retrieve_results['available']:
            if result['downloadId'] in preparingDownloadIds:
                preparingDownloadIds.remove(result['downloadId'])
                runDownload(threads, result['url'])
                print(f"       {result['url']}\n" )
            
        for result in download_retrieve_results['requested']:   
            if result['downloadId'] in preparingDownloadIds:
                preparingDownloadIds.remove(result['downloadId'])
                runDownload(threads, result['url'])
                print(f"       {result['url']}\n" )
    
    # Didn't get all download URLs, retrieve again after 30 seconds
    while len(preparingDownloadIds) > 0: 
        print(f"{len(preparingDownloadIds)} downloads are not available yet. Waiting for 30s to retrieve again\n")
        time.sleep(30)
        download_retrieve_results = sendRequest(serviceUrl + "download-retrieve", download_ret_payload, headers, False)
        if download_retrieve_results != False:
            for result in download_retrieve_results['available']:                            
                if result['downloadId'] in preparingDownloadIds:
                    preparingDownloadIds.remove(result['downloadId'])
                    print(f"    Get download url: {result['url']}\n" )
                    runDownload(threads, result['url'])
                    
print("\nDownloading files... Please do not close the program\n")
for thread in threads:
    thread.join()

Get download url: https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/004/053/LC08_L2SP_004053_20210516_20210525_02_T1/LC08_L2SP_004053_20210516_20210525_02_T1_SR_B3.TIF?requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MTM0NjcyNiwiZG93bmxvYWRJZCI6NjU0NTIzMTA0LCJkYXRlR2VuZXJhdGVkIjoiMjAyNC0wOC0wNVQyMzowMzoxNy0wNTowMCIsImlkIjoiTEMwOF9MMlNQXzAwNDA1M18yMDIxMDUxNl8yMDIxMDUyNV8wMl9UMV9TUl9CMy5USUYiLCJzaWduYXR1cmUiOiIkNSQkbFlMZEpBRVlldE1NSlJHTzdcL3kxSG5Xai56WW45TFIxQXEuc2o1U0pRS0MifQ==

Get download url: https://landsatlook.usgs.gov/data/collection02/level-2/standard/oli-tirs/2021/004/053/LC08_L2SP_004053_20210516_20210525_02_T1/LC08_L2SP_004053_20210516_20210525_02_T1_SR_B5.TIF?requestSignature=eyJkb3dubG9hZEFwcCI6Ik0yTSIsImNvbnRhY3RJZCI6MTM0NjcyNiwiZG93bmxvYWRJZCI6NjU0NTIzMTA2LCJkYXRlR2VuZXJhdGVkIjoiMjAyNC0wOC0wNVQyMzowMzoxNy0wNTowMCIsImlkIjoiTEMwOF9MMlNQXzAwNDA1M18yMDIxMDUxNl8yMDIxMDUyNV8wMl9UMV9TUl9CNS5USUYiLCJzaWduYXR1cmUiOiIkNSQkbjFCenRpV1VaWXo3R0pL

KeyboardInterrupt: 

    Downloading: LC08_L2SP_004053_20220111_20220123_02_T1_SR_B5.TIF...
    Downloading: LC08_L2SP_004053_20220503_20220511_02_T1_SR_B3.TIF...


## **9. Remove the scene list with [*scene-list-remove*](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-remove)**<a id="scenelistremove"></a>
Use [***scene-list-remove***](https://m2m.cr.usgs.gov/api/docs/reference/#scene-list-remove) to delete items from a specified list, a ***`listId`*** is required. Since no ***`datasetName`*** is provided, the entire list is removed. This is important as scene-lists are preserved between sessions.

In [ ]:
# remove_scnlst_payload = {
#     "listId": listId
# }
# sendRequest(serviceUrl + "scene-list-remove", remove_scnlst_payload, headers)

# if fileType == 'band_group':    
#     # Remove the secondary scene list
#     remove_scnlst2_payload = {
#         "listId": secondaryListId
#     }
#     sendRequest(serviceUrl + "scene-list-remove", remove_scnlst2_payload, headers)

## **10. [*Logout*](https://m2m.cr.usgs.gov/api/docs/reference/#logout)**<a id="logout"></a>
Logout so the API Key cannot be used anymore.

<div class="alert alert-block alert-warning">
    <h4> <b>NOTE:</b> The Machine-to-Machine API key expires after 2 hours of inactivity. </h4>
</div>  

In [ ]:
# endpoint = "logout"  
# if sendRequest(serviceUrl + endpoint, None, apiKey) == None:        
#     print("\nLogged Out\n")
# else:
#     print("\nLogout Failed\n")

## **11. List Downloads**

In [ ]:
os.listdir(data_dir)


<div class="alert alert-block alert-info">
    <h1> Contact Information </h1>
    <h3> Material written by Tonian Robinson<sup>1</sup> </h3>
    <ul>
        <b>Contact:</b> custserv@usgs.gov <br> 
        <b>Voice:</b> +1-605-594-6151 <br>
        <b>Organization:</b> USGS EROS User Services <br>
        <b>Date last modified:</b> 01-Jul-2024 <br>
    </ul>
    
<sup>1</sup>Earth Space Technology Services LLC., contractor to the U.S. Geological Survey, Earth Resources Observation and Science (EROS) Center, Sioux Falls, South Dakota, 57198-001, USA. Work performed under USGS contract G0121D0001.
</div>